<a href="https://colab.research.google.com/github/AmitNativ1984/Autobrains-Unet-Cityscapes/blob/main/Algo_DNN_Home_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TASK 1 - Semantic Segmentation using U-net

Identifying the drivable area provides critical information for navigating and path planning in autonomous driving. In this task you will take the first step torward this feature by segmenting the road in the image.


# **Save Your Time**
Please first download the data. The data is transfered via Google drive which somtimes slow.
*   [Zip file - containing the data](https://drive.google.com/file/d/1_Demk2hTXuVPq9bBToedS4fOLqwaG7N3/view?usp=share_link)
*   [Google drive with all the files](https://drive.google.com/drive/folders/10udfdPAT0yPq1TA7Ld0ddSKMedbEJBoe?usp=share_link)

![](https://raw.githubusercontent.com/henyau/Image-Segmentation-with-Unet/master/images/train_label.png)


You are provided with a partial training code of a U-net model. Your task is to train a model that predicts the road segment in the image. Please implement the missing components of the training pipeline, train the model and preform evaluation.

### Data - road and vehicle segmentation

The zip file provided contains images and high quality dense pixel annotations (fine annotation). The data is split into train (\~2900 images) and validation (\~500 images) with the directories similar the original Cityscapes dataset. 

The original Cityscapes anotations contains 19 classes. Please reduce the number of classes from 19 to the following 3: “background”, “vehicle”, “road”.You can read more about Cityscape here https://www.cityscapes-dataset.com/dataset-overview/#labeling-policy.


The Zip file provides with the following:
1.   cityscapes dir - a directory with the images and annotations. Make sure the annotations contains the 3 classes mentioned above
2.   image_list dir - a tsv file for train and validation. Each row in the tsv is a pairs of paths, path to image and path to annotation. In each TSV change **/PATH_TO_CITYSCAPES_DATA_DIR/** to the absolute path of cityscapes dir



To save time:
*   Resize images to a small fixed-size that will enable you to train the model in a fesiable time.

# **Deliverables**


1.   Complete the training code for road segmentation (marked "COMPLETE ME"):
  * **Loss function** (Training Flow section)
  * **Optimizer** (Training Flow section)
  * **Augmentations** (Data preparations section)
2.   **Perform an evaluation of your trained model** (Evaluation section)


### Trainer Class

In [37]:
import numpy as np
import torch
from PIL import Image
import os
import random
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch import Tensor

In [38]:
class Trainer:
    def __init__(self,
                 model: torch.nn.Module,
                 device: torch.device,
                 criterion: torch.nn.Module,
                 optimizer: torch.optim.Optimizer,
                 training_DataLoader: torch.utils.data.Dataset,
                 validation_DataLoader: torch.utils.data.Dataset = None,
                 lr_scheduler: torch.optim.lr_scheduler = None,
                 epochs: int = 100,
                 epoch: int = 0,
                 notebook: bool = False
                 ):

        self.model = model
        self.criterion = criterion.cuda()
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.training_DataLoader = training_DataLoader
        self.validation_DataLoader = validation_DataLoader
        self.device = device
        self.epochs = epochs
        self.epoch = epoch
        self.notebook = notebook

        self.training_loss = []
        self.validation_loss = []
        self.learning_rate = []

        self.best_val_loss = np.inf
        self.best_dice_score = 0

        self.tb_writer = SummaryWriter(comment="_lr_{}".format(self.optimizer.param_groups[0]['lr']))

        # create dir for saving model
        if not os.path.exists("models"):
            os.mkdir("models")

        self.model_path = "models/"

    def run_trainer(self):

        if self.notebook:
            from tqdm.notebook import tqdm, trange
        else:
            from tqdm import tqdm, trange

        progressbar = trange(self.epochs, desc='Progress')
        for i in progressbar:
            """Epoch counter"""
            self.epoch += 1  # epoch counter

            """Training block"""
            self._train()

            """Validation block"""
            if self.validation_DataLoader is not None:
                self._validate()

            """Learning rate scheduler block"""
            if self.lr_scheduler is not None:
                if self.validation_DataLoader is not None and self.lr_scheduler.__class__.__name__ == 'ReduceLROnPlateau':
                    self.lr_scheduler.step(
                        self.validation_loss[i])  # learning rate scheduler step with validation loss
                else:
                    self.lr_scheduler.step()  # learning rate scheduler step
        return self.training_loss, self.validation_loss, self.learning_rate

    def _train(self):

        if self.notebook:
            from tqdm.notebook import tqdm, trange
        else:
            from tqdm import tqdm, trange

        self.model.train()  # train mode
        train_losses = []  # accumulate the losses here
        batch_iter = tqdm(enumerate(self.training_DataLoader), 'Training', total=len(self.training_DataLoader),
                          leave=False)
        for i, (x, y) in batch_iter:
            input, target = x.to(self.device), y.to(self.device)  # send to device (GPU or CPU)
            self.optimizer.zero_grad()  # zerograd the parameters
            out = self.model(input)  # one forward pass
            loss = self.criterion(out, target.long())  # calculate loss
            loss_value = loss.item()
            train_losses.append(loss_value)
            loss.backward()  # one backward pass
            self.optimizer.step()  # update the parameters

            if i % 10 == 0:
                self.write_predictions_to_tensorboard(out, 'train_predictions')

            batch_iter.set_description(f'Training: (loss {loss_value:.4f})')  # update progressbar
        print("TRAIN MEAN LOSS: {}".format(np.mean(train_losses)))
        self.training_loss.append(np.mean(train_losses))
        self.learning_rate.append(self.optimizer.param_groups[0]['lr'])

        self.tb_writer.add_scalar('Loss/train', np.mean(train_losses), self.epoch)
        self.tb_writer.add_scalar('Learning rate', self.optimizer.param_groups[0]['lr'], self.epoch)

        batch_iter.close()

    def _validate(self):

        if self.notebook:
            from tqdm.notebook import tqdm, trange
        else:
            from tqdm import tqdm, trange

        self.model.eval()  # evaluation mode
        valid_losses = []  # accumulate the losses here
        batch_iter = tqdm(enumerate(self.validation_DataLoader), 'Validation', total=len(self.validation_DataLoader),
                          leave=False)

        dice_scores = []

        img2save = random.randint(0, len(self.validation_DataLoader))
        for i, (x, y) in batch_iter:
            input, target = x.to(self.device), y.to(self.device)  # send to device (GPU or CPU)

            with torch.no_grad():
                out = self.model(input)
                loss = self.criterion(out, target.long())
                loss_value = loss.item()
                valid_losses.append(loss_value)

                batch_iter.set_description(f'Validation: (loss {loss_value:.4f})')
        
            # add images to tensorboard every 10 batches
            if i == img2save:
               self.write_predictions_to_tensorboard(out, 'val_predictions')

            # evaluate dice score
            # convert to one-hot format
            target_19classes = target.clone()
            target_19classes[target == 255] = 18
            mask_onehot = F.one_hot(target_19classes.long(), 19).permute(0, 3, 1, 2).float()
            pred_onehot = F.one_hot(out.argmax(dim=1), 19).permute(0, 3, 1, 2).float()
            # compute the Dice score, averaged over only the first 3 classes
            dice_batch = self.multiclass_dice_coeff(pred_onehot[:, :3], mask_onehot[:, :3], reduce_batch_first=False)
            dice_scores.append(dice_batch.cpu().numpy())

        self.validation_loss.append(np.mean(valid_losses))
        self.tb_writer.add_scalar('Loss/val', np.mean(valid_losses), self.epoch)
        print("VAL MEAN LOSS: {}".format(np.mean(valid_losses)))

        self.tb_writer.add_scalar('Dice/val', np.mean(dice_scores), self.epoch)
        print("VAL MEAN DICE SCORE: {}".format(np.mean(dice_scores)))

        # saving current model:
        torch.save(self.model.state_dict(), self.model_path + "last.pth".format(self.epoch))

        # save checkpoint every 5 epochs
        if self.epoch % 5 == 0:
            torch.save(self.model.state_dict(), self.model_path + "checkpoint_{}.pth".format(self.epoch))

        if np.mean(dice_scores) > self.best_dice_score:
            print("NEW BEST DICE SCORE: {}, SAVING MODEL".format(np.mean(dice_scores)))
            self.best_dice_score = np.mean(dice_scores)
            torch.save(self.model.state_dict(), self.model_path + "best.pth")
        
        
        batch_iter.close()

    def decode_pred(self, pred):
        # Put all void classes to zero
        
        labeled_pred = np.zeros((pred.shape[0], pred.shape[1], 3), dtype=np.uint8)
        labeled_pred[pred == 0,:] = [0, 0, 255]    #road
        labeled_pred[pred == 1,:] = [255, 255, 0]  # background
        labeled_pred[pred == 2,:] = [0, 255, 0]  # vehicle
        
        return labeled_pred

    def write_predictions_to_tensorboard(self, out, tb_label):
         # convert out to labels in color
            pred = torch.softmax(out, dim=1)
            pred = torch.argmax(out, dim=1)
            pred = pred.cpu().detach().numpy()
            pred = np.array([self.decode_pred(x) for x in pred])
            
            # pred = torch.from_numpy(pred).float()
            
            # convert pred to tensor
            pred = torch.from_numpy(pred).float()
            
            # concatenate images to one image
            pred = torch.cat([pred[i] for i in range(pred.shape[0])], dim=1)

            
            # pred = pred.view(pred.shape[1], -1, 3)
            
            self.tb_writer.add_image(tb_label, pred.numpy().astype(np.uint8), global_step=self.epoch, dataformats='HWC')
            self.tb_writer.flush()
            
        

    def dice_coeff(self, input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon: float = 1e-6):
        # Average of Dice coefficient for all batches, or for a single mask
        assert input.size() == target.size()
        assert input.dim() == 3 or not reduce_batch_first

        sum_dim = (-1, -2) if input.dim() == 2 or not reduce_batch_first else (-1, -2, -3)

        inter = 2 * (input * target).sum(dim=sum_dim)
        sets_sum = input.sum(dim=sum_dim) + target.sum(dim=sum_dim)
        sets_sum = torch.where(sets_sum == 0, inter, sets_sum)

        dice = (inter + epsilon) / (sets_sum + epsilon)
        return dice.mean()


    def multiclass_dice_coeff(self, input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon: float = 1e-6):
        # Average of Dice coefficient for all classes
        return self.dice_coeff(input.flatten(0, 1), target.flatten(0, 1), reduce_batch_first, epsilon)

###  Data preparations

#### Augmentations

In [39]:
import os
import cv2
import torch
import random
import numbers
import numpy as np
from PIL import Image, ImageOps, ImageFilter
from torchvision import transforms

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample: tuple) -> tuple:
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img, mask = sample
        
        img, mask = self.resize(img, mask, (128, 128))

        img = np.array(img).astype(np.float32).transpose((2, 0, 1))
        mask = np.array(mask).astype(np.float32)
                
        img = torch.from_numpy(img).float()
        mask = torch.from_numpy(mask).float()

        img = self.normalize(img)
        
        return img, mask

# Add your augmentations here
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# ----------------------------"COMPLETE ME"------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------

    def normalize(self, img: torch.Tensor) -> torch.Tensor:
        # Normalize the image
        normalize = [
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ]
        
        img = transforms.Compose(normalize)(img)
        return img

    def resize(self, img, mask, size: tuple):
        
        # Resize
        resize_img = transforms.Resize(size, interpolation=transforms.InterpolationMode.BICUBIC)
        resize_mask = transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST)
                
        # Resize the image
        img = transforms.Compose([resize_img])(img)
        mask = transforms.Compose([resize_mask])(mask)
        return img, mask

#### Dataset object

In [40]:
from torch.utils import data
from torchvision import transforms

class TrainDataset(data.Dataset):
    """
    Dataset for semantic segmentation. Data is stored as image list. 
    An image list file is a tsv file where each row contains the path to an image and its gt path.
    """
    NUM_CLASSES = 19

    def __init__(self, img_list_path: str, split="train"):
        """
        :param args: dataset args for training
        :param img_list_path: path to the directory of image list files
        :param split: type of dataset, train test or validation
        """
        self.split = split
        self.files = {}
        self.gts = {}
        imgs = []
        gts = []

        # Parsing the image_list  
        for dirpath, dirnames, filenames in os.walk(os.path.join(img_list_path, split)):
            for file in filenames:
                with open(os.path.join(dirpath, file), mode='r') as img_list:
                    for row in img_list:
                        _, img, gt = row.split('\t')
                        imgs.append(img)
                        gts.append(gt.strip())

        self.files[split] = imgs
        self.gts[split] = gts

        # Cityscapes classes 
        self.void_classes = [0, 1, 2, 3, 4, 5, 6, 9, 10, 14, 15, 16, 18, 29, 30, -1]
        self.valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
        self.class_names = ['unlabelled', 'road', 'sidewalk', 'building', 'wall', 'fence', \
                            'pole', 'traffic_light', 'traffic_sign', 'vegetation', 'terrain', \
                            'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train', \
                            'motorcycle', 'bicycle']

        self.ignore_index = 255
                
        self.set_augmentation_type()

        if not self.files[split]:
            raise Exception("No files for split=[%s] found in %s" % (split, img_list_path))

        print("Found %d %s images" % (len(self.files[split]), split))

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        img_path = self.files[self.split][index].rstrip()
        lbl_path = self.gts[self.split][index].rstrip()

        _img = np.array(Image.open(img_path).convert('RGB'))
        _target = np.array(Image.open(lbl_path), dtype=np.uint8)

        _img = Image.fromarray(_img)
        _target = Image.fromarray(_target)

        sample = (_img, _target)
        if self.split == 'train':
            return self.transform_train(sample)
        elif self.split == 'val':
            return self.transform_validation(sample)
        elif self.split == 'test':
            return self.transform_test(sample)

    def encode_segmap(self, mask):
        # Put all void classes to zero
        for _voidc in self.void_classes:
            mask[mask == _voidc] = self.ignore_index
        for _validc in self.valid_classes:
            mask[mask == _validc] = self.class_map[_validc]
        return mask

    def set_seed(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
       
    
    def transform_tr(self, sample):                              
        resize_img = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.BICUBIC)]
        resize_mask = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.NEAREST)]
        
        geometric_transforms = [
            transforms.RandomApply(torch.nn.ModuleList([
                                    transforms.CenterCrop((96, 192)),
                                    ]), p=0.2),
            transforms.RandomHorizontalFlip(p=0.5),
        ]

        color_transforms = [
            transforms.RandomApply(torch.nn.ModuleList([
                                    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
                                    ]),p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
        ]
        
        state = torch.get_rng_state()
        img = sample[0]
        img_transforms = transforms.Compose(resize_img + geometric_transforms + color_transforms)
        img_aug = img_transforms(img)

        torch.set_rng_state(state)
        mask = sample[1]
        mask_transforms = transforms.Compose(resize_mask + geometric_transforms)
        mask_aug = mask_transforms(mask)

        sample = (img_aug, mask_aug)

        composed_transforms = transforms.Compose([ToTensor()])
        return composed_transforms(sample)

    def transform_val(self, sample):
        resize_img = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.BICUBIC)]
        resize_mask = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.NEAREST)]

        img = sample[0]
        img_transforms = transforms.Compose(resize_img)
        img_aug = img_transforms(img)

        mask = sample[1]
        mask_transforms = transforms.Compose(resize_mask)
        mask_aug = mask_transforms(mask)

        sample = (img_aug, mask_aug)
        composed_transforms = transforms.Compose([ToTensor()])
        return composed_transforms(sample)

    def transform_ts(self, sample):
        resize_img = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.BICUBIC)]
        resize_mask = [transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.NEAREST)]

        img = sample[0]
        img_transforms = transforms.Compose(resize_img)
        img_aug = img_transforms(img)

        mask = sample[1]
        mask_transforms = transforms.Compose(resize_mask)
        mask_aug = mask_transforms(mask)

        sample = (img_aug, mask_aug)
        composed_transforms = transforms.Compose([ToTensor()])
        return composed_transforms(sample)

    def get_img_list(self):
        img_list = [(img_path, os.path.join(self.annotations_base,
                                            img_path.split(os.sep)[-2],
                                            os.path.basename(img_path)[:-15] + 'gtFine_labelIds.png')) for img_path in
                    self.files[self.split]]
        return img_list

    def set_augmentation_type(self):
        self.transform_train = self.transform_tr
        self.transform_validation = self.transform_val
        self.transform_test = self.transform_ts


### Training Flow

In [41]:
!pip install unet

In [42]:
import pathlib
import torch

import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from skimage.transform import resize
from unet import UNet

from tqdm.notebook import tqdm, trange

In [43]:
img_list_dir = "/DATA/img_list"
# For example: img_list_dir = "/content/drive/MyDrive/Algo_Home_Assignment/Cityscapes/img_list"
batch_size = 8


In [44]:
train_set = TrainDataset(img_list_path=img_list_dir, split='train')
val_set = TrainDataset(img_list_path=img_list_dir, split='val')
dataloader_training = DataLoader(train_set, batch_size=batch_size, drop_last=True, shuffle=True)
dataloader_validation = DataLoader(val_set, batch_size=1, drop_last=True, shuffle=False)

Found 2975 train images
Found 500 val images


In [45]:
class Config():
  def __init__(self) -> None:
      self.EPOCHS = 50
      # lr
      self.lr = 0.001 # so far the best was 0.0001
      self.momentum = 0.9
      
      # lr scheduler
      self.multistep_milestones = [40, 45]
      self.lr_gamma = 0.1

In [46]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU")
else:
    print("CPU")
    torch.device('cpu')

# model
model = UNet(in_channels=3,
             out_classes=19,
             out_channels_first_layer=64,
             num_encoding_blocks=2,
             padding=2,
             dimensions=2).to(device)

# -----------------------------------------------------------------------------
# ---------------------------------  "COMPLETE ME"  ---------------------------
config = Config()

# ~~~~~~~~~~~~~~~~~~~~ FIND CLASS WEIGHTS ~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~ (was done once, and the results are hardcoded below) ~~~~~
# 
# count occrrences of each class in the training set for loss weights calculation
# class_occurences = np.zeros(19)
# batch_iter = tqdm(enumerate(dataloader_training), 'Calculating class appearances', total=len(dataloader_training),
#                           leave=False)
# for b, batch in batch_iter:
#     for i in range(batch[1].shape[0]):
#         for j in range(19):
#             class_occurences[j] += torch.sum(batch[1][i] == j).item()
#
# num of pixels per class was found to be: [63426380, 94339816, 14138511]
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

num_of_pixels_per_class = np.zeros(19)
num_of_pixels_per_class[:3] = np.array([63426380, 94339816, 14138511])

valid_classes_weights = 1/(num_of_pixels_per_class[:3]) / sum(1/(num_of_pixels_per_class[:3]))
# [0.16, 0.11. 0.73] -- ["road", "background", "vehicle"]
#weight more for less frequent classes
class_weights = np.ones(19)
class_weights[:3] = valid_classes_weights
class_weights[3:] = 1

criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights).float().to(device), ignore_index=255)
# criterion = torch.nn.CrossEntropyLoss(ignore_index=255)


## Add an optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)


## Add a learning rate scheduler 
lr_scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=config.multistep_milestones, gamma=config.lr_gamma)

# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------

# # load model
# checkpoint = torch.load('models/best.pth')
# model.load_state_dict(checkpoint)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=lr_scheduler,
                  epochs=config.EPOCHS,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()


GPU


TRAIN MEAN LOSS: 1.1430017099868899


VAL MEAN LOSS: 0.8990612605810165
VAL MEAN DICE SCORE: 0.3054979741573334
NEW BEST DICE SCORE: 0.3054979741573334, SAVING MODEL


TRAIN MEAN LOSS: 0.873220566951361


VAL MEAN LOSS: 0.9012338931560516
VAL MEAN DICE SCORE: 0.3173447251319885
NEW BEST DICE SCORE: 0.3173447251319885, SAVING MODEL


...

VAL MEAN LOSS: 0.3938956789970398
VAL MEAN DICE SCORE: 0.6792674660682678


TRAIN MEAN LOSS: 0.42630744815836696


VAL MEAN LOSS: 0.3909153105914593
VAL MEAN DICE SCORE: 0.6833296418190002


TRAIN MEAN LOSS: 0.42976466291194976


VAL MEAN LOSS: 0.39521636775135993
VAL MEAN DICE SCORE: 0.6776052713394165



GOOD LUCK !